# Question 1

Given is a six-armed bandit, as introduced in the lecture.

- The first arm shall sample its reward uniformly from the interval $[-1,4)$.
- The second arm shall sample its reward uniformly from $[2,6)$.
- The third arm shall sample its reward uniformly from the interval $[-2,3)$.
- The fourth arm shall sample its reward uniformly from $[5,9)$.
- The fifth arm shall sample its reward uniformly from $[-3,5)$.
- The sixth arm shall sample its reward uniformly from $[1,6)$.

What is the expected reward when actions are chosen uniformly?

For a **uniform distribution** $\mathcal{U}(a,b)$, the *expected value* (mean) is
given by:

$$
  \mathbb{E}(X) = (a + b) / 2
$$

This is because the uniform distribution is symmetric - it's "center"
(midpoint) is its average. So when we want to compute the expected reward of
pulling, say Arm<sub>1</sub> which gives a reward from $\mathcal{U}(-1,4)$, the average
reward over time (expected reward) is:

$$
  \mathbb{E}(\text{``Reward"}) = \frac{-1 + 4}{2} = 1.5
$$

Applying this to each arm:

- Arm<sub>1</sub>: $[-1, 4) \Rightarrow \frac{-1+4}{2} = 1.5$
- Arm<sub>2</sub>: $[ 2, 6) \Rightarrow \frac{2+6}{2}  = 4$
- Arm<sub>3</sub>: $[-2, 3) \Rightarrow \frac{-2+3}{2} = 0.5$
- Arm<sub>4</sub>: $[ 5, 9) \Rightarrow \frac{5+9}{2}  = 7$
- Arm<sub>5</sub>: $[-3, 5) \Rightarrow \frac{-3+5}{2} = 1$
- Arm<sub>6</sub>: $[ 1, 6) \Rightarrow \frac{1+6}{2}  = 3.5$

Since actions are chosen **uniformly**, the overall expected reward is the
average of the expected values:

$$
  \frac{1}{6}(1.5+4+0.5+7+1+3.5) = \frac{17.5}{6} \approx 2.9167
$$

# Question 2

Implement the six-armed bandit from [question 1](#question-1) and compute the
sample average reward for 20 uniformly chosen actions!

Compare this to your expectation from [question 1](#question-1)!

In [1]:
import random

K = 20
ARM_REWARDS = (
    (-1, 4),
    ( 2, 6),
    (-2, 3),
    ( 5, 9),
    (-3, 5),
    ( 1, 6),
)

averages = 0
for _ in range(K):
    actions = random.choices(ARM_REWARDS, k=K)
    reward_sum = sum(
        map(
            lambda r: random.uniform(*r),
            actions,
        )
    )
    averages += reward_sum / K

print(averages/K)


2.8569101398809464


The output of this program quickly approximates to the expected output we had in
[Question 1](#question-1).

# Question 3

Initialize $Q(a_i)=0$ and chose $2000$ actions according to an
$\varepsilon$-greedy selection strategy ($\varepsilon=0.1$)!

Update your action values by computing the sample average reward of each
action recursively!

For every $100$ actions show the percentage of choosing arm<sub>1</sub>,
arm<sub>2</sub>, arm<sub>3</sub>, arm<sub>4</sub>, arm<sub>5</sub>, and
arm<sub>6</sub> as well as the resulting average reward!

In [2]:
import random

K = 2000
PATCH = 100
EPSILON = 0.1
ARM_REWARDS = (
    (-1.0, 4.0),
    ( 2.0, 6.0),
    (-2.0, 3.0),
    ( 5.0, 9.0),
    (-3.0, 5.0),
    ( 1.0, 6.0),
)
ideal_actions = [lower for lower, _ in ARM_REWARDS]
action_count = [0 for _ in ARM_REWARDS]

def new_average(old_average: float, step: int, reward: float) -> float:
    return old_average + 1/(step+1) * (reward - old_average)

def update_actions(idx: int, reward: float, step: int) -> float:
    """Updates ``ideal_actions`` and ``action_count`` with the best
    reward/count"""
    ideal_actions[idx] = new_average(ideal_actions[idx], step, reward)
    action_count[idx] += 1
    return reward

def exploit(step: int) -> float:
    """Choose the best known action so far"""
    i, ideal = 0, ideal_actions[0]
    for j, v in enumerate(ideal_actions[1:], start=1):
        if v > ideal:
            ideal = v
            i = j
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward, step)

def explore(step: int) -> float:
    """Choose a random action to perform"""
    i = random.choice(range(len(ARM_REWARDS)))
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward, step)

overall_average = 0
for i in range(K//PATCH):
    actions = random.choices(
        (exploit, explore),
        weights=(1-EPSILON, EPSILON),
        k=PATCH,
    )
    for j, action in enumerate(actions, start=1):
        step = i * PATCH + j
        reward = action(step)
        overall_average = new_average(overall_average, step, reward)

    tokens = []
    for j, count in enumerate(action_count, start=1):
        per = 100 * count / ((i+1) * PATCH)
        tokens.append(f"arm({j})={per:.2f}%")
    print(" ".join(tokens))
    print(f"overall average = {overall_average:.2f}", end="\n\n")

arm(1)=0.00% arm(2)=0.00% arm(3)=0.00% arm(4)=98.00% arm(5)=1.00% arm(6)=1.00%
overall average = 6.91

arm(1)=0.00% arm(2)=1.50% arm(3)=0.00% arm(4)=95.00% arm(5)=1.50% arm(6)=2.00%
overall average = 6.86

arm(1)=1.67% arm(2)=1.00% arm(3)=0.67% arm(4)=93.67% arm(5)=1.67% arm(6)=1.33%
overall average = 6.76

arm(1)=1.50% arm(2)=1.00% arm(3)=0.75% arm(4)=93.00% arm(5)=2.00% arm(6)=1.75%
overall average = 6.66

arm(1)=1.20% arm(2)=2.00% arm(3)=1.00% arm(4)=92.40% arm(5)=1.80% arm(6)=1.60%
overall average = 6.63

arm(1)=1.33% arm(2)=1.67% arm(3)=1.67% arm(4)=91.33% arm(5)=2.17% arm(6)=1.83%
overall average = 6.56

arm(1)=1.57% arm(2)=1.43% arm(3)=1.71% arm(4)=91.71% arm(5)=1.86% arm(6)=1.71%
overall average = 6.58

arm(1)=1.50% arm(2)=1.62% arm(3)=1.50% arm(4)=92.00% arm(5)=1.88% arm(6)=1.50%
overall average = 6.60

arm(1)=1.78% arm(2)=1.67% arm(3)=1.56% arm(4)=91.33% arm(5)=2.22% arm(6)=1.44%
overall average = 6.57

arm(1)=1.80% arm(2)=1.70% arm(3)=1.70% arm(4)=91.30% arm(5)=2.20% arm(6)=

Which also agrees with the results show in [Question 1](#question-1), since
Arm<sub>4</sub> has the highest overall average and since we're spending $90\%$
of our trials exploiting the algorithm quickly learns to always choose from
Arm<sub>4</sub>, raising the overall average to a whopping increase in the
overall average rather than what we had in [Question 2](#question-2).

It's alive 🙂

# Question 4

Redo the experiment, but after $1000$ steps sample the rewards of the fourth
arm uniformly from $[-4,3)$!

Compare updating action values by computing the sample average reward of each
action recursively (as done in [question 3](#question-3)) with using a constant
learning rate $\alpha=0.05$!

For every $100$ actions show the percentage of choosing arm<sub>1</sub>,
arm<sub>2</sub>, arm<sub>3</sub>, arm<sub>4</sub>, arm<sub>5</sub>, and
arm<sub>6</sub> as well as the resulting average reward!

In [3]:
import random

K = 2000
PATCH = 100
EPSILON = 0.1
ARM_REWARDS = [
    (-1.0, 4.0),
    ( 2.0, 6.0),
    (-2.0, 3.0),
    ( 5.0, 9.0),
    (-3.0, 5.0),
    ( 1.0, 6.0),
]
ideal_actions = [lower for lower, _ in ARM_REWARDS]
action_count = [0 for _ in ARM_REWARDS]

ALPHA = 0.05
def new_average(old_average: float, reward: float) -> float:
    return old_average + ALPHA * (reward - old_average)

def update_actions(idx: int, reward: float) -> float:
    """Updates ``ideal_actions`` and ``action_count`` with the best
    reward/count"""
    ideal_actions[idx] = new_average(ideal_actions[idx], reward)
    action_count[idx] += 1
    return reward

def exploit() -> float:
    """Choose the best known action so far"""
    i, ideal = 0, ideal_actions[0]
    for j, v in enumerate(ideal_actions[1:], start=1):
        if v > ideal:
            ideal = v
            i = j
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward)

def explore() -> float:
    """Choose a random action to perform"""
    i = random.choice(range(len(ARM_REWARDS)))
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward)

overall_average = 0
for i in range(K//PATCH):
    actions = random.choices(
        (exploit, explore),
        weights=(1-EPSILON, EPSILON),
        k=PATCH,
    )
    for j, action in enumerate(actions, start=1):
        reward = action()
        overall_average = new_average(overall_average, reward)

    tokens = []
    for j, count in enumerate(action_count, start=1):
        per = 100 * count / ((i+1) * PATCH)
        tokens.append(f"arm({j})={per:.2f}%")
    print(" ".join(tokens))
    print(f"overall average = {overall_average:.2f}", end="\n\n")

    if i == 9:
        ARM_REWARDS[3] = (-4,3)


arm(1)=2.00% arm(2)=3.00% arm(3)=3.00% arm(4)=91.00% arm(5)=1.00% arm(6)=0.00%
overall average = 6.41

arm(1)=1.00% arm(2)=2.00% arm(3)=3.00% arm(4)=93.00% arm(5)=0.50% arm(6)=0.50%
overall average = 6.96

arm(1)=1.33% arm(2)=2.00% arm(3)=2.67% arm(4)=91.67% arm(5)=1.00% arm(6)=1.33%
overall average = 6.84

arm(1)=1.00% arm(2)=1.75% arm(3)=2.00% arm(4)=92.00% arm(5)=1.75% arm(6)=1.50%
overall average = 6.56

arm(1)=1.20% arm(2)=2.20% arm(3)=1.80% arm(4)=91.80% arm(5)=1.60% arm(6)=1.40%
overall average = 6.65

arm(1)=1.17% arm(2)=2.33% arm(3)=1.83% arm(4)=91.33% arm(5)=1.83% arm(6)=1.50%
overall average = 6.07

arm(1)=1.29% arm(2)=2.29% arm(3)=1.71% arm(4)=91.29% arm(5)=1.86% arm(6)=1.57%
overall average = 6.73

arm(1)=1.62% arm(2)=2.25% arm(3)=1.88% arm(4)=90.88% arm(5)=1.88% arm(6)=1.50%
overall average = 6.46

arm(1)=1.56% arm(2)=2.11% arm(3)=1.78% arm(4)=91.44% arm(5)=1.78% arm(6)=1.33%
overall average = 6.72

arm(1)=1.90% arm(2)=2.00% arm(3)=1.80% arm(4)=91.20% arm(5)=1.70% arm(6)=

Which shows the algorithms adaptability since during midway Arm<sub>4</sub> isn't
no longer the action with the best overall average ($\mathbb{E}(\text{Arm}_4) =
\frac{-4 + 3}{2} = -0.5$) the second best canidate according to [Question
1](#question-1) is Arm<sub>2</sub> with an overall average of $4$, which
explains the increase in Arm<sub>2</sub> usage.

It's alive 🙂

# Question 5

Modify the experiment from [question 4](#question-4) by using an optimistic
initialization $Q(a_i)=10$ and a greedy action selection strategy, still using a
constant learning rate $\alpha=0.05$!

For every $100$ actions show the percentage of choosing arm<sub>1</sub>,
arm<sub>2</sub>, arm<sub>3</sub>, arm<sub>4</sub>, arm<sub>5</sub>, and
arm<sub>6</sub> as well as the resulting average reward!

Compare this to your result from [question 4](#question-4)!

In [4]:
import random

K = 2000
PATCH = 100
EPSILON = 0.1
ARM_REWARDS = [
    (-1.0, 4.0),
    ( 2.0, 6.0),
    (-2.0, 3.0),
    ( 5.0, 9.0),
    (-3.0, 5.0),
    ( 1.0, 6.0),
]
ideal_actions = [10.0 for _ in ARM_REWARDS]
action_count = [0 for _ in ARM_REWARDS]

ALPHA = 0.05
def new_average(old_average: float, reward: float) -> float:
    return old_average + ALPHA * (reward - old_average)

def update_actions(idx: int, reward: float) -> float:
    """Updates ``ideal_actions`` and ``action_count`` with the best
    reward/count"""
    ideal_actions[idx] = new_average(ideal_actions[idx], reward)
    action_count[idx] += 1
    return reward

def exploit() -> float:
    """Choose the best known action so far"""
    i, ideal = 0, ideal_actions[0]
    for j, v in enumerate(ideal_actions[1:], start=1):
        if v > ideal:
            ideal = v
            i = j
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward)

def explore() -> float:
    """Choose a random action to perform"""
    i = random.choice(range(len(ARM_REWARDS)))
    reward = random.uniform(*ARM_REWARDS[i])
    return update_actions(i, reward)

overall_average = 0
for i in range(K//PATCH):
    actions = random.choices(
        (exploit, explore),
        weights=(1-EPSILON, EPSILON),
        k=PATCH,
    )
    for j, action in enumerate(actions, start=1):
        reward = action()
        overall_average = new_average(overall_average, reward)

    tokens = []
    for j, count in enumerate(action_count, start=1):
        per = 100 * count / ((i+1) * PATCH)
        tokens.append(f"arm({j})={per:.2f}%")
    print(" ".join(tokens))
    print(f"overall average = {overall_average:.2f}", end="\n\n")

    if i == 9:
        ARM_REWARDS[3] = (-4,3)


arm(1)=9.00% arm(2)=13.00% arm(3)=8.00% arm(4)=47.00% arm(5)=10.00% arm(6)=13.00%
overall average = 5.47

arm(1)=5.00% arm(2)=7.50% arm(3)=4.50% arm(4)=69.50% arm(5)=6.00% arm(6)=7.50%
overall average = 6.67

arm(1)=4.33% arm(2)=5.67% arm(3)=3.33% arm(4)=76.67% arm(5)=4.33% arm(6)=5.67%
overall average = 6.67

arm(1)=3.75% arm(2)=4.50% arm(3)=3.25% arm(4)=79.75% arm(5)=4.00% arm(6)=4.75%
overall average = 6.31

arm(1)=3.40% arm(2)=3.80% arm(3)=2.80% arm(4)=82.60% arm(5)=3.40% arm(6)=4.00%
overall average = 6.57

arm(1)=3.50% arm(2)=3.17% arm(3)=2.67% arm(4)=83.83% arm(5)=3.17% arm(6)=3.67%
overall average = 6.48

arm(1)=3.14% arm(2)=3.14% arm(3)=2.29% arm(4)=85.29% arm(5)=3.00% arm(6)=3.14%
overall average = 6.69

arm(1)=3.00% arm(2)=3.00% arm(3)=2.25% arm(4)=86.12% arm(5)=2.62% arm(6)=3.00%
overall average = 6.42

arm(1)=2.89% arm(2)=3.11% arm(3)=2.11% arm(4)=86.67% arm(5)=2.44% arm(6)=2.78%
overall average = 6.57

arm(1)=2.60% arm(2)=3.20% arm(3)=2.30% arm(4)=87.00% arm(5)=2.30% arm(

Which does exhibit a similar behaviour to [Question 4](#question-4) answers,
(i.e. it starts exploit Arm<sub>2</sub> midways) with an increase in the overall
average and an increase in the overall usage of different arms (aka
exploration).